In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#baixar a vercis recente do spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

In [ ]:
#Deszipar o spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

In [ ]:
#Criar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=a271d002604472555118de49062104a17127f8081fb2082a4bdd00f9d601b5d0
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
#Instalar a lib findspark que ajuda a localizar o Spark no sistema e importá-lo como uma biblioteca regular
!pip install -q findspark

In [ ]:
#importar a lib findspark
import findspark
findspark.init()

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

In [ ]:
# Instalar ipython-sql
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00


In [ ]:
# Carregar a extensão SQL
%load_ext sql

#Criar um SQLite database
%sql sqlite://

In [ ]:
# %%sql serve para multiplas linhas de código
%sql sqlite:///stroke_data

In [ ]:
!pip install gdown


In [ ]:
import gdown

url = "https://drive.google.com/uc?id=163BGU_RzXR29UbVVkPpv8tYnUejlPBVr"
output = "stroke_data.csv"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=163BGU_RzXR29UbVVkPpv8tYnUejlPBVr
To: /content/stroke_data.csv
100%|██████████| 4.16M/4.16M [00:00<00:00, 86.1MB/s]


'stroke_data.csv'

In [ ]:
# Configurar Spark
spark = SparkSession.builder.appName("Stroke_data").getOrCreate()

In [ ]:
income_df = spark.read.csv('stroke_data.csv',header='True',inferSchema='true')
income_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [ ]:
income_df.show(10)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

In [ ]:
income_df.describe('gender').show()

+-------+------+
|summary|gender|
+-------+------+
|  count| 67135|
|   mean|  null|
| stddev|  null|
|    min|Female|
|    max| Other|
+-------+------+



In [ ]:
from pyspark.ml.feature import VectorAssembler

#Usa colunas numéricas para previsão
numericCols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]

#VectorAssembler é um transformador
#Transforma um df com colunas em um vetor com colunas
vecAssembler = VectorAssembler(inputCols=numericCols, outputCol="features")

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Crie uma instância do DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

In [ ]:
from pyspark.ml.feature import StringIndexer

# Use StringIndexer para converter a coluna 'stroke' em um valor numérico
labelToIndex = StringIndexer(inputCol="stroke", outputCol="stroke_label")

In [ ]:
from pyspark.ml.feature import StringIndexer

# Use StringIndexer para converter a coluna 'stroke' em um valor numérico
labelToIndex = StringIndexer(inputCol="stroke", outputCol="stroke_label")
income_df = labelToIndex.fit(income_df).transform(income_df)

In [ ]:
from pyspark.ml import Pipeline

# Um pipeline é uma sequência de estágios
dtc = DecisionTreeClassifier(labelCol='stroke_label', featuresCol='features')
pipeline = Pipeline(stages=[vecAssembler, dtc])

# Separa os dados em dados de treinamento e teste
train_data, test_data = income_df.randomSplit([0.7, 0.3])

# Pipeline é um estimador que recebe um DataFrame e produz um modelo.
pipelineModel = pipeline.fit(train_data)

# Aplica o modelo do pipeline aos dados de teste
predictionsDF = pipelineModel.transform(test_data)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Acurácia: % de previsão corretas
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='stroke_label')
print(f"Acurácia:{evaluator.evaluate(predictionsDF)}")

Acurácia:0.6865300437723836


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categoricalCols = []

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

#StringIndexer para converter 'gender' e 'smoking_status' em valores numéricos:
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
smoking_indexer = StringIndexer(inputCol="smoking_status", outputCol="smoking_status_index")

#OneHotEncoder para codificar as colunas indexadas:
gender_encoder = OneHotEncoder(inputCol="gender_index", outputCol="gender_encoded")
smoking_encoder = OneHotEncoder(inputCol="smoking_status_index", outputCol="smoking_status_encoded")

#Atualize a lista numericCols para incluir as colunas numéricas originais, bem como as colunas codificadas:
numericCols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level", "gender_encoded", "smoking_status_encoded"]

#Atualize o VectorAssembler para incluir as novas colunas:
vecAssembler = VectorAssembler(inputCols=numericCols, outputCol="features")

#Atualize o pipeline para incluir as etapas de indexação e codificação:
pipeline = Pipeline(stages=[gender_indexer, smoking_indexer, gender_encoder, smoking_encoder, vecAssembler, dtc])


In [ ]:
# Separa os dados em dados de treinamento e teste
train_data, test_data = income_df.randomSplit([0.7, 0.3])

# Pipeline é um estimador que recebe um DataFrame e produz um modelo.
pipelineModel = pipeline.fit(train_data)

# Aplica o modelo do pipeline aos dados de teste
predictionsDF = pipelineModel.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Acurácia: % de previsão corretas
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='stroke_label')
accuracy = evaluator.evaluate(predictionsDF)
print(f"Acurácia: {accuracy}")


Acurácia: 0.8373882820258193


In [ ]:
# Obter o modelo de árvore de decisão do pipeline
dt_model = pipelineModel.stages[-1]

# Criar um DataFrame com as importâncias das características
importance_df = pd.DataFrame({'Feature': all_feature_cols, 'Importance': dt_model.featureImportances.toArray()})

# Classificar as características pela importância em ordem decrescente
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Exibir a importância das características
print(importance_df)



                  Feature  Importance
7          gender_encoded    0.490593
8  smoking_status_encoded    0.334985
0                     age    0.167821
4       avg_glucose_level    0.005689
1                     bmi    0.000911
2            hypertension    0.000000
3           heart_disease    0.000000
5          gender_encoded    0.000000
6  smoking_status_encoded    0.000000


In [ ]:
# Obter o modelo de árvore de decisão do pipeline
dt_model = pipelineModel.stages[-1]

# Obter a profundidade da árvore
tree_depth = dt_model.getMaxDepth()

# Exibir a profundidade da árvore
print("Profundidade da Árvore de Decisão:", tree_depth)


Profundidade da Árvore de Decisão: 5


In [ ]:
# Obter o modelo de árvore de decisão do pipeline
dt_model = pipelineModel.stages[-1]

# Obter o número de nós na árvore
num_nodes = dt_model.numNodes

# Exibir o número de nós na árvore
print("Número de Nós na Árvore de Decisão:", num_nodes)


Número de Nós na Árvore de Decisão: 17
